In [18]:
# https://pypi.org/project/exif/
# pip install exif

# https://pypi.org/project/pymediainfo/
# pip install pymediainfo

In [68]:
import pandas as pd
# import PIL.Image
from exif import Image
from pprint import pprint
from pymediainfo import MediaInfo
from pathlib import Path
import numpy as np

In [20]:
media_info = MediaInfo.parse("VID_20190106_162804.mp4")
for track in media_info.tracks:
    if track.track_type == "Video":
        exif_vdata = track.to_data()
exif_vdata['tagged_date']

'UTC 2019-01-07 00:28:08'

In [22]:
with open('IMG_20211123_162603.jpg', 'rb') as image_file:
    my_image = Image(image_file)
my_image.has_exif

True

In [30]:
# список всех методов
# my_image.list_all()

In [24]:
my_image.datetime

'2021:11:23 16:26:05'

In [25]:
my_image.datetime_original

'2021:11:23 16:26:05'

In [26]:
my_image.datetime_digitized

'2021:11:23 16:26:05'

In [27]:
my_image.gps_datestamp

'2021:11:23'

In [32]:
path = Path('D:/Рабочая/!_Scripts/change_file_creation_time')

In [53]:
df = pd.DataFrame(columns=['name', 'extension', 'path'])
for file_path in path.glob('**/*.*'):
    extension = str(file_path).split('.')[-1]
    name = file_path.name
    parent_path = file_path.parent
    df.loc[len(df)] = [name, extension, parent_path]
df

,name,extension,path
0,.ipynb_checkpoints,ipynb_checkpoints,D:\Рабочая\!_Scripts\change_file_creation_time
1,changeFileCreationTime.py,py,D:\Рабочая\!_Scripts\change_file_creation_time
2,IMG_20211123_162603.jpg,jpg,D:\Рабочая\!_Scripts\change_file_creation_time
3,IMG_20211123_162801.jpg,jpg,D:\Рабочая\!_Scripts\change_file_creation_time
4,tmp.py,py,D:\Рабочая\!_Scripts\change_file_creation_time
5,Untitled.ipynb,ipynb,D:\Рабочая\!_Scripts\change_file_creation_time
6,VID_20190106_162804.mp4,mp4,D:\Рабочая\!_Scripts\change_file_creation_time
7,Untitled-checkpoint.ipynb,ipynb,D:\Рабочая\!_Scripts\change_file_creation_time...


In [56]:
# посмотрим сколько каких файлов по типар расширения у нас есть
df.extension.value_counts()

jpg                  2
py                   2
ipynb                2
mp4                  1
ipynb_checkpoints    1
Name: extension, dtype: int64

In [69]:
# списки расширений для видео и фото
video_ext = np.array(['mp4'])
photo_ext = np.array(['jpg'])

In [70]:
# df.query('extension in @video_ext or extension in @photo_ext')

,name,extension,path
2,IMG_20211123_162603.jpg,jpg,D:\Рабочая\!_Scripts\change_file_creation_time
3,IMG_20211123_162801.jpg,jpg,D:\Рабочая\!_Scripts\change_file_creation_time
6,VID_20190106_162804.mp4,mp4,D:\Рабочая\!_Scripts\change_file_creation_time
